# Nested cross-validation structure with feature selection using LASSO

In [1]:
import pandas as pd
import numpy as np
import collections
import scipy
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.calibration import calibration_curve
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectPercentile, mutual_info_classif
from sklearn.feature_selection import f_regression
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import auc
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import matthews_corrcoef
from scipy import interp
from statistics import stdev
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
import warnings
warnings.filterwarnings("ignore")
import time
import pickle

In [2]:
df = pd.read_csv("../../data/harmonized/combined_harmonized.csv", sep="\t")

df2 = pd.read_csv("../../data/clinical/clinical_data_internal_combined.csv", sep=";", index_col=0)

In [3]:
df2.drop(df2.columns[1:6], axis=1, inplace=True)
df2.drop(df2.columns[2], axis=1, inplace=True)
df = df2.join(df.set_index('ID_intern'), on='ID_intern')
df = df.rename({'Pathology': 'Type'}, axis=1)

In [4]:
df

,ID_intern,Type,T1_original_shape_Elongation,T1_original_shape_Flatness,T1_original_shape_LeastAxisLength,T1_original_shape_MajorAxisLength,T1_original_shape_Maximum2DDiameterColumn,T1_original_shape_Maximum2DDiameterRow,T1_original_shape_Maximum2DDiameterSlice,T1_original_shape_Maximum3DDiameter,...,T1fs_original_gldm_DependenceVariance,T1fs_original_gldm_GrayLevelNonUniformity,T1fs_original_gldm_GrayLevelVariance,T1fs_original_gldm_HighGrayLevelEmphasis,T1fs_original_gldm_LargeDependenceEmphasis,T1fs_original_gldm_LargeDependenceLowGrayLevelEmphasis,T1fs_original_gldm_LowGrayLevelEmphasis,T1fs_original_gldm_SmallDependenceEmphasis,T1fs_original_gldm_SmallDependenceHighGrayLevelEmphasis,T1fs_original_gldm_SmallDependenceLowGrayLevelEmphasis
ID,,,,,,,,,,,,,,,,,,,,,
0,LIPO_002,0,0.250806,0.139088,0.225144,0.486140,0.189037,0.336436,0.463397,0.447998,...,0.392114,0.037301,0.021410,0.059376,0.217081,0.029379,0.069781,0.051594,0.001241,0.019483
1,LIPO_006,0,0.956647,0.839193,0.903002,0.535274,0.619874,0.565238,0.570082,0.563262,...,0.116511,0.142997,0.075332,0.159306,0.058250,-0.002527,0.025307,0.171700,0.050888,0.030947
2,LIPO_007,1,0.486505,0.467856,0.291966,0.276583,0.175388,0.272151,0.268000,0.259158,...,0.290507,0.022492,0.073235,0.056163,0.117083,0.030527,0.107069,0.156891,0.031479,0.062275
3,LIPO_009,0,0.865762,0.639992,0.944379,0.710976,0.708308,0.661475,0.680350,0.697248,...,0.312078,0.219517,0.104173,0.196247,0.137075,0.001853,0.016859,0.120955,0.069555,0.003478
4,LIPO_010,1,0.501202,0.383670,0.370937,0.414304,0.417894,0.402481,0.210499,0.388404,...,0.580909,0.122396,0.010837,0.037791,0.334436,0.078517,0.110207,0.039649,0.000776,0.014309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,LT202,0,0.610670,0.148018,0.067588,0.162248,0.103619,0.162699,0.162354,0.152985,...,0.767144,0.004832,0.012005,0.027382,0.415566,0.151211,0.197115,0.057972,0.004302,0.066838
151,LT203,0,0.758654,0.267662,0.083371,0.130821,0.123474,0.126999,0.111748,0.125729,...,0.295447,0.012926,0.023644,0.039880,0.149361,0.053821,0.157847,0.136706,0.021127,0.127816
152,LT208,1,0.261232,0.376160,0.228461,0.269606,0.132136,0.188281,0.251662,0.251105,...,0.109881,0.014791,0.138150,0.272585,0.065258,0.010759,0.031444,0.206859,0.096794,0.041305


In [5]:
tumor_ids = df['ID_intern']

In [6]:
tumor_ids

ID
0      LIPO_002
1      LIPO_006
2      LIPO_007
3      LIPO_009
4      LIPO_010
         ...   
150       LT202
151       LT203
152       LT208
153       LT214
154       LT215
Name: ID_intern, Length: 155, dtype: object

In [7]:
# class imbalance
alts = len(df[df['Type'] == 1])
lipomas = len(df) - alts
total = len(df)
print(f'{str("{:.0f}".format(lipomas/total*100))}:{str("{:.0f}".format(alts/total*100))} negative to positive ratio')

80:20 negative to positive ratio


3:1 negative to positive ratio

In [8]:
features = list(df.columns[2:])

y = df['Type']
y = pd.DataFrame.to_numpy(y) # data object: numpy array

X = df.drop(df.columns[[0, 1]], axis=1)
X = pd.DataFrame.to_numpy(X) # data object: numpy array

In [ ]:
def plotCM(cm, i, loop, ct_o, ct_i, k):
    display = ConfusionMatrixDisplay(confusion_matrix=cm)
    display.plot()
    plt.title("Confusion matrix " + r"$\bf{" + loop + "}$" + " loop")
    plt.savefig(f'../../results/lasso/combined/lasso-ncv{i}-cm-{loop}-o{ct_o}i{ct_i}-alpha-{k}.png')
    plt.close()

In [ ]:
def mrmr(xTrain, xTest, yTrain, given_features, K):
    #    X: pandas.DataFrame, features
    #    y: pandas.Series, target variable
    #    K: number of features to select
    
    xTrain = pd.DataFrame(data=xTrain, columns=given_features)
    
    # only to filter it at the end
    xTest = pd.DataFrame(data=xTest, columns=given_features)
    
    # for F-statistic
    y = pd.Series(yTrain)

    # compute F-statistics and initialize correlation matrix
    F = pd.Series(f_regression(xTrain, y)[0], index = xTrain.columns)
    corr = pd.DataFrame(.00001, index = xTrain.columns, columns = xTrain.columns)
    
    # initialize list of selected features and list of excluded features
    selected = []
    not_selected = xTrain.columns.to_list()
    
    # repeat K times
    for i in range(K):
        # compute (absolute) correlations between the last selected feature and all the (currently) excluded features
        if i > 0:
            last_selected = selected[-1]
            corr.loc[not_selected, last_selected] = xTrain[not_selected].corrwith(xTrain[last_selected]).abs().clip(.00001)
            
        # compute FCQ score for all the (currently) excluded features (this is Formula 2)
        score = F.loc[not_selected] / corr.loc[not_selected, selected].mean(axis = 1).fillna(.00001)
        
        # find best feature, add it to selected and remove it from not_selected
        best = score.index[score.argmax()]
        selected.append(best)
        not_selected.remove(best)
        
    # filter columns
    xTrain_filtered = xTrain.drop(not_selected, axis = 1)
    xTest_filtered = xTest.drop(not_selected, axis = 1)

    
    # convert back to numpy array
    xTrain_filtered = pd.DataFrame.to_numpy(xTrain_filtered)
    xTest_filtered = pd.DataFrame.to_numpy(xTest_filtered)
    
    # print("!!!!", xTrain_filtered.shape, xTest_filtered.shape)
        
    return xTrain_filtered, xTest_filtered, selected, not_selected

In [ ]:
def pca_vis(X_train_i, features):
        X_train_i = pd.DataFrame(data=X_train_i, columns=features)
        tot_var = 0.95 # total variance
        pca_model = PCA(n_components = tot_var)
        
        X_train_pca = PCA(tot_var, svd_solver = 'full').fit(X_train_i)
        
        # print("Variance ratio:", X_train_pca.explained_variance_ratio_); print()
        # print("PCA dimensions:", X_train_pca.components_.shape[0]); print()
        # print("Reduced dimensions can explain {:.4f}".format(sum(X_train_pca.explained_variance_ratio_)*100),
        #       "% of the variance in the original data."); print()
        
        # components
        n_pcs= X_train_pca.components_.shape[0]
        
        # PCA coverts the features in array format; so, if we want to get the feature names:
        # most_important = [np.abs(X_train_pca.components_[i]).argmax() for i in range(n_pcs)]
        # most_important_names = [features[most_important[i]] for i in range(n_pcs)]
        # dic = {'PC{}'.format(i): most_important_names[i] for i in range(n_pcs)}
        # p = pd.DataFrame(dic.items())
        # # print("New dimensions:\n", p); print()
        # 
        # X_train_i = pd.DataFrame.to_numpy(X_train_i) # data object: numpy array
        
        return n_pcs

In [ ]:
f = open("../../results/lasso/combined/lasso-output.txt", "w")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


def plot_learning_curve(
    train_sizes_ncv,
    train_scores_mean,
    train_scores_std,
    test_scores_mean,
    test_scores_std,
    fit_times_mean,
    fit_times_std,
    axes=None,
    ylim=None,
    cv=None,
    n_jobs=None,
    train_sizes=np.linspace(0.1, 1.0, 5),
):
    """
    Generate 3 plots: the test and training learning curve, the training
    samples vs fit times curve, the fit times vs score curve.

    Parameters
    ----------
    estimator : estimator instance
        An estimator instance implementing `fit` and `predict` methods which
        will be cloned for each validation.

    title : str
        Title for the chart.

    X : array-like of shape (n_samples, n_features)
        Training vector, where ``n_samples`` is the number of samples and
        ``n_features`` is the number of features.

    y : array-like of shape (n_samples) or (n_samples, n_features)
        Target relative to ``X`` for classification or regression;
        None for unsupervised learning.

    axes : array-like of shape (3,), default=None
        Axes to use for plotting the curves.

    ylim : tuple of shape (2,), default=None
        Defines minimum and maximum y-values plotted, e.g. (ymin, ymax).

    cv : int, cross-validation generator or an iterable, default=None
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:

          - None, to use the default 5-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, default=None
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like of shape (n_ticks,)
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the ``dtype`` is float, it is regarded
        as a fraction of the maximum size of the training set (that is
        determined by the selected validation method), i.e. it has to be within
        (0, 1]. Otherwise it is interpreted as absolute sizes of the training
        sets. Note that for classification the number of samples usually have
        to be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title("LASSO Learning Curve")
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    
    # train_sizes, train_scores, test_scores, fit_times, _ = learning_curve(
    #     estimator,
    #     X,
    #     y,
    #     cv=cv,
    #     n_jobs=n_jobs,
    #     train_sizes=train_sizes,
    #     return_times=True,
    # )
    # train_scores_mean = np.mean(train_scores, axis=1)
    # train_scores_std = np.std(train_scores, axis=1)
    # test_scores_mean = np.mean(test_scores, axis=1)
    # test_scores_std = np.std(test_scores, axis=1)
    # fit_times_mean = np.mean(fit_times, axis=1)
    # fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(
        train_sizes_ncv,
        train_scores_mean - train_scores_std,
        train_scores_mean + train_scores_std,
        alpha=0.1,
        color="b",
    )
    axes[0].fill_between(
        train_sizes_ncv,
        test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std,
        alpha=0.1,
        color="g",
    )
    axes[0].plot(
        train_sizes_ncv, train_scores_mean, "o-", color="b", label="Training score"
    )
    axes[0].plot(
        train_sizes_ncv, test_scores_mean, "o-", color="g", label="Cross-validation score"
    )
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes_ncv, fit_times_mean, "o-")
    axes[1].fill_between(
        train_sizes_ncv,
        fit_times_mean - fit_times_std,
        fit_times_mean + fit_times_std,
        alpha=0.1,
    )
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    fit_time_argsort = fit_times_mean.argsort()
    fit_time_sorted = fit_times_mean[fit_time_argsort]
    test_scores_mean_sorted = test_scores_mean[fit_time_argsort]
    test_scores_std_sorted = test_scores_std[fit_time_argsort]
    axes[2].grid()
    axes[2].plot(fit_time_sorted, test_scores_mean_sorted, "o-")
    axes[2].fill_between(
        fit_time_sorted,
        test_scores_mean_sorted - test_scores_std_sorted,
        test_scores_mean_sorted + test_scores_std_sorted,
        alpha=0.1,
    )
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt

In [ ]:
# configure the nested CV procedure
# dataset D=(X, y)

# define number of folds based on dataset size and keep folds while looping through different models
K1 = 3 # outer
K2 = 3 # inner

# define the model
model = SGDClassifier(penalty='elasticnet', l1_ratio=1, loss='log') # log loss because hinge does not allow predict_proba

# define a search space with at least 2 parameters so we have a 2D grid space
space = dict()
space['alpha'] = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100] # learning rate
# space['n_iter'] = [1, 5, 10]

# GridSearchCV custom implementation
series = range(1, 51) # for multiple runs of CV

# metrics across all 150 models of the 50 nCV runs
accuracy_ncv  = []
balanced_accuracy_ncv  = []  
f1_ncv  = []
recall_ncv  = []
mcc_ncv = []
sensitivity_ncv = []
specificity_ncv = []

hyperparameters = []
hp_features = []

mean_accuracy = {}
mean_balanced_accuracy = {}
mean_f1 = {}
mean_recall = {}
mean_mcc = {}

# ROC -------------------------------
tprs = []
aucs = []
base_fpr = np.linspace(0, 1, 101)
colors = ['darkorange']

fig, ax = plt.subplots()
# -----------------------------------

# calibration curve -----------------
probs_true = []
probs_pred = []
# -----------------------------------

# learning curve --------------------
train_sizes_ncv = []
train_scores_ncv = []
test_scores_ncv = []
fit_times_ncv = []
# -----------------------------------

# models to file
g = open('/Users/dianadavid/Uni/Thesis/repo/results/lasso/combined/lasso_final_models_combined.dat', 'wb')


start_time = time.time()
for i in series:
    cv_outer = StratifiedKFold(n_splits=K1, random_state=i, shuffle=True)
    cv_inner = StratifiedKFold(n_splits=K2, random_state=i, shuffle=True)

    # print(f'\n\033[1m NESTED CV RUN #{i} \033[0m')
    f.write(f'\n\033[1m NESTED CV RUN #{i} \033[0m')
    
    # outer loop metrics
    accuracy_outer_folds  = []
    balanced_accuracy_outer_folds  = []  
    f1_outer_folds  = []
    recall_outer_folds  = []
    mcc_outer_folds = []
    
    outer_hp = {}

    # OUTER LOOP
    ct_o = 0
    for train_indices, test_indices in cv_outer.split(X, y):
        ct_o = ct_o + 1
        f.write(f'\n----------- Outer loop #{ct_o} -----------')
        
        # inner loop metrics
        best_score = 0.0
        best_hp = 0.0001
        best_feature_set = []
        balanced_accuracy_inner_folds  = []
        
        selected_features = []
        # not_selected_features = []
        
        # print("Train indices: ", train_indices, "\nTest indices: ", test_indices, "\n\n")
        X_train_i, X_test_i = X[train_indices], X[test_indices]
        y_train_i, y_test_i = y[train_indices], y[test_indices]
                
        ct_i = 0
        # INNER LOOP
        for train_indices_inner, test_indices_inner in cv_inner.split(X_train_i, y_train_i):
            ct_i = ct_i + 1
            f.write(f'\nInner loop #{ct_i}')
            
            # print("Train indices inner: ", train_indices_inner, "\nTest indices inner: ", test_indices_inner, "\n\n")
            X_train_j, X_test_j = X_train_i[train_indices_inner], X_train_i[test_indices_inner]
            y_train_j, y_test_j = y_train_i[train_indices_inner], y_train_i[test_indices_inner]
            
            # PCA visualisation ------------------------------------------------------------------------------------
            dimensions = pca_vis(X_train_j, features)
            # ------------------------------------------------------------------------------------------------------

            # MRMR
            X_train_j, X_test_j, selected_inner_features, not_selected_inner_features = mrmr(X_train_j, X_test_j, y_train_j, features, dimensions)
            
            # over- and undersampling
            over = SMOTE(sampling_strategy=0.5)
            under = RandomUnderSampler(sampling_strategy=0.6)
            steps = [('over', over), ('under', under)]
            pipeline = Pipeline(steps=steps)
            X_train_j, y_train_j = pipeline.fit_resample(X_train_j, y_train_j)
            
            f.write(f'\n# of selected inner features: {len(selected_inner_features)}')
            selected_features.append(selected_inner_features) # list of lists to compare
            
            # not_selected_features.append(not_selected_inner_features)
   
            # --------------------------------------------------------------------------------------------------------
            
            for k in space['alpha']:
                    model.alpha = k
                    model.fit(X_train_j, y_train_j)

                    y_predicted_inner_test = model.predict(X_test_j)

                    score = balanced_accuracy_score(y_test_j, y_predicted_inner_test)
            
                    if score > best_score:
                        best_score = score
                        best_hp = k
                        best_feature_set = selected_inner_features
             
                    # cm_i = confusion_matrix(y_test_j, y_predicted_inner_test)
                    # plotCM(cm_i, i, "inner", ct_o, ct_i, k)
            
            f.write("\n------------------------------------ end of inner loop ------------------------------------")
          
        # feature selection ------------------------------------------------------------------------------------        
        X_train_i = pd.DataFrame(data=X_train_i, columns=features)
        X_test_i = pd.DataFrame(data=X_test_i, columns=features)
        
        cols = [col for col in X_train_i.columns if col in best_feature_set]
        
        X_train_i = X_train_i[cols]
        X_test_i = X_test_i[cols]
        
        X_train_i = pd.DataFrame.to_numpy(X_train_i)
        X_test_i = pd.DataFrame.to_numpy(X_test_i)
        # ------------------------------------------------------------------------------------------------------
        
        # set params (best item from inner)
        model.alpha = best_hp
        
        hyperparameters.append(best_hp)
        hp_features.append(best_feature_set)
        
        model.fit(X_train_i, y_train_i)
        
        pickle.dump(model, g)
        # print("\n", model)

        y_predicted_outer_test = model.predict(X_test_i)
        
        # metrics
        accuracy_outer_folds.append(accuracy_score(y_test_i, y_predicted_outer_test))
        balanced_accuracy_outer_folds.append(balanced_accuracy_score(y_test_i, y_predicted_outer_test))
        f1_outer_folds.append(f1_score(y_test_i, y_predicted_outer_test))
        recall_outer_folds.append(recall_score(y_test_i, y_predicted_outer_test))
        mcc_outer_folds.append(matthews_corrcoef(y_test_i, y_predicted_outer_test))
        
        cm_o = confusion_matrix(y_test_i, y_predicted_outer_test)
        # plotCM(cm_o, i, "outer", ct_o, '', 0)
        # print("!!!!", cm_o[0,0], cm_o[0,1], cm_o[1,0], cm_o[1,1])
        
        sensitivity_o = cm_o[1,1]/(cm_o[1,1]+cm_o[1,0]) # TP/(TP+TN)
        f.write(f'\nSensitivity: {round(sensitivity_o*100, 2)}%')

        specificity_o = cm_o[0,0]/(cm_o[0,0]+cm_o[0,1]) # TN/(TN+FP)
        f.write(f'\nSpecificity: {round(specificity_o*100, 2)}%')
        
        sensitivity_ncv.append(sensitivity_o)
        specificity_ncv.append(specificity_o)
        
        # ROC ------------------------------------------------------------------------------------------------
        y_score = model.predict_proba(X_test_i)
        fpr, tpr, _ = roc_curve(y_test_i, y_score[:, 1])
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        #plt.plot(fpr, tpr, lw=1, alpha=0.6, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc), c = colors[i])
        tpr = interp(base_fpr, fpr, tpr)
        tpr[0] = 0.0
        tprs.append(tpr)
        # ----------------------------------------------------------------------------------------------------
        
        # calibration curve ----------------------------------------------------------------------------------
        prob_true, prob_pred = calibration_curve(y_test_i, y_score[:,1], strategy='quantile')
        probs_true.append(prob_true)
        probs_pred.append(prob_pred)
        # ----------------------------------------------------------------------------------------------------
        
        # learning curve -------------------------------------------------------------------------------------
        train_sizes, train_scores, test_scores, fit_times, _ = learning_curve(
            model,
            X_train_i,
            y_train_i,
            cv=cv_outer,
            n_jobs=4,
            train_sizes=np.linspace(0.1, 1.0, 5),
            return_times=True,
        )
        train_sizes_ncv.append(train_sizes)
        train_scores_ncv.append(train_scores)
        test_scores_ncv.append(test_scores)
        fit_times_ncv.append(fit_times)
        # ------------------------------------------------------------------------------------------------

        
        # check how many features were selected in all loops
        f.write(f'# common features: {len(set(selected_features[0]) & set(selected_features[1]) & set(selected_features[2]))}')
        
        f.write("\n------------------------------------ end of outer loop ------------------------------------")
    


    f.write(f'\nBest accuracy: {best_score}')
    f.write(f'\nBest alpha: {model.alpha}')
    
    accuracy_ncv.append(accuracy_outer_folds)
    balanced_accuracy_ncv.append(balanced_accuracy_outer_folds)
    f1_ncv.append(f1_outer_folds)
    recall_ncv.append(recall_outer_folds)
    mcc_ncv.append(mcc_outer_folds)

    mean_accuracy['Nested CV run #' + str(i)] = [str("{:.2f}".format(np.mean(np.array(accuracy_outer_folds)))) +  " +/- " + str("{:.2f}".format(np.std(np.array(accuracy_outer_folds))))]
    mean_balanced_accuracy['Nested CV run #' + str(i)] = [str("{:.2f}".format(np.mean(np.array(balanced_accuracy_outer_folds)))) +  " +/- " + str("{:.2f}".format(np.std(np.array(balanced_accuracy_outer_folds))))]
    mean_f1['Nested CV run #' + str(i)] = [str("{:.2f}".format(np.mean(np.array(f1_outer_folds)))) +  " +/- " + str("{:.2f}".format(np.std(np.array(f1_outer_folds))))]
    mean_recall['Nested CV run #' + str(i)] = [str("{:.2f}".format(np.mean(np.array(recall_outer_folds)))) +  " +/- " + str("{:.2f}".format(np.std(np.array(recall_outer_folds))))]
    mean_mcc['Nested CV run #' + str(i)] = [str("{:.2f}".format(np.mean(np.array(mcc_outer_folds)))) +  " +/- " + str("{:.2f}".format(np.std(np.array(mcc_outer_folds))))]
    
    f.write("\n---------------------------------- end of nested cv run -----------------------------------")

g.close()

ff = open('/Users/dianadavid/Uni/Thesis/repo/results/lasso/combined/lasso_features_combined.txt', 'w')
for item in hp_features:
    ff.write(str(item) + "\n")
ff.close()

gg = open('/Users/dianadavid/Uni/Thesis/repo/results/lasso/combined/lasso_hyperparams_combined.txt', 'w')
for item in hyperparameters:
    gg.write(str(item) + "\n")
gg.close()
         
# learning curve ---------------------------------------------------------------------------------------------
train_sizes_ncv = np.mean(train_sizes_ncv, axis=0)
train_sizes_ncv = np.around(train_sizes_ncv)
train_sizes_ncv = train_sizes_ncv.astype(int)

train_scores_mean = []
train_scores_std = []
tr_sc1 = []
tr_sc2 = []
tr_sc3 = []
tr_sc4 = []
tr_sc5 = []

test_scores_mean = []
test_scores_std = []
t_sc1 = []
t_sc2 = []
t_sc3 = []
t_sc4 = []
t_sc5 = []

fit_times_mean = []
fit_times_std = []
f_sc1 = []
f_sc2 = []
f_sc3 = []
f_sc4 = []
f_sc5 = []

for i in range(len(train_scores_ncv)-1):
    tr_sc1.append(train_scores_ncv[i][0])
    tr_sc2.append(train_scores_ncv[i][1])
    tr_sc3.append(train_scores_ncv[i][2])
    tr_sc4.append(train_scores_ncv[i][3])
    tr_sc5.append(train_scores_ncv[i][4])
    
    t_sc1.append(test_scores_ncv[i][0])
    t_sc2.append(test_scores_ncv[i][1])
    t_sc3.append(test_scores_ncv[i][2])
    t_sc4.append(test_scores_ncv[i][3])
    t_sc5.append(test_scores_ncv[i][4])
    
    f_sc1.append(fit_times_ncv[i][0])
    f_sc2.append(fit_times_ncv[i][1])
    f_sc3.append(fit_times_ncv[i][2])
    f_sc4.append(fit_times_ncv[i][3])
    f_sc5.append(fit_times_ncv[i][4])  
    
tr_sc1 = np.ravel(tr_sc1)
tr_sc1_std = np.std(tr_sc1)
tr_sc1 = np.mean(tr_sc1) 

train_scores_mean.append(tr_sc1)
train_scores_std.append(tr_sc1_std)

tr_sc2 = np.ravel(tr_sc2)
tr_sc2_std = np.std(tr_sc2)
tr_sc2 = np.mean(tr_sc2) 

train_scores_mean.append(tr_sc2)
train_scores_std.append(tr_sc2_std)

tr_sc3 = np.ravel(tr_sc3)
tr_sc3_std = np.std(tr_sc3)
tr_sc3 = np.mean(tr_sc3) 

train_scores_mean.append(tr_sc3)
train_scores_std.append(tr_sc3_std)

tr_sc4 = np.ravel(tr_sc4)
tr_sc4_std = np.std(tr_sc4)
tr_sc4 = np.mean(tr_sc4) 

train_scores_mean.append(tr_sc4)
train_scores_std.append(tr_sc4_std)

tr_sc5 = np.ravel(tr_sc5)
tr_sc5_std = np.std(tr_sc5)
tr_sc5 = np.mean(tr_sc5) 

train_scores_mean.append(tr_sc5)
train_scores_mean = np.array(train_scores_mean)
train_scores_std.append(tr_sc5_std)
train_scores_std = np.array(train_scores_std)
    
# print("Train scores mean: ", train_scores_mean) 
# print("Train scores std: ", train_scores_std) 

t_sc1 = np.ravel(t_sc1)
t_sc1_std = np.std(t_sc1)
t_sc1 = np.mean(t_sc1) 

test_scores_mean.append(t_sc1)
test_scores_std.append(t_sc1_std)

t_sc2 = np.ravel(t_sc2)
t_sc2_std = np.std(t_sc2)
t_sc2 = np.mean(t_sc2) 

test_scores_mean.append(t_sc2)
test_scores_std.append(t_sc2_std)

t_sc3 = np.ravel(t_sc3)
t_sc3_std = np.std(t_sc3)
t_sc3 = np.mean(t_sc3) 

test_scores_mean.append(t_sc3)
test_scores_std.append(t_sc3_std)

t_sc4 = np.ravel(t_sc4)
t_sc4_std = np.std(t_sc4)
t_sc4 = np.mean(t_sc4) 

test_scores_mean.append(t_sc4)
test_scores_std.append(t_sc4_std)

t_sc5 = np.ravel(t_sc5)
t_sc5_std = np.std(t_sc5)
t_sc5 = np.mean(t_sc5) 

test_scores_mean.append(t_sc5)
test_scores_mean = np.array(test_scores_mean)
test_scores_std.append(t_sc5_std)
test_scores_std = np.array(test_scores_std)
    
# print("Test scores mean: ", test_scores_mean) 
# print("Test scores std: ", test_scores_std) 

f_sc1 = np.ravel(f_sc1)
f_sc1_std = np.std(f_sc1)
f_sc1 = np.mean(f_sc1) 

fit_times_mean.append(f_sc1)
fit_times_std.append(f_sc1_std)

f_sc2 = np.ravel(f_sc2)
f_sc2_std = np.std(f_sc2)
f_sc2 = np.mean(f_sc2) 

fit_times_mean.append(f_sc2)
fit_times_std.append(f_sc2_std)

f_sc3 = np.ravel(f_sc3)
f_sc3_std = np.std(f_sc3)
f_sc3 = np.mean(f_sc3) 

fit_times_mean.append(f_sc3)
fit_times_std.append(f_sc3_std)

f_sc4 = np.ravel(f_sc4)
f_sc4_std = np.std(f_sc4)
f_sc4 = np.mean(f_sc4) 

fit_times_mean.append(f_sc4)
fit_times_std.append(f_sc4_std)

f_sc5 = np.ravel(f_sc5)
f_sc5_std = np.std(f_sc5)
f_sc5 = np.mean(f_sc5) 

fit_times_mean.append(f_sc5)
fit_times_mean = np.array(fit_times_mean)
fit_times_std.append(f_sc5_std)
fit_times_std = np.array(fit_times_std)
    
# print("Fit times mean: ", fit_times_mean) 
# print("Fit times std: ", fit_times_std) 
    
plot_learning_curve(train_sizes_ncv, train_scores_mean, train_scores_std, test_scores_mean, test_scores_std, 
                    fit_times_mean, fit_times_std, cv=cv_outer, n_jobs=4)
plt.savefig(f'../../results/lasso/combined/ncv-lasso-learning-curve.png')
plt.close()
# -----------------------------------------------------------------------------------------------------------

# calibration curve -----------------------------------------------------------------------------------------
fig, ax = plt.subplots()
probs_pred = np.array(probs_pred)
probs_true = np.array(probs_true)
probs_pred = probs_pred.mean(axis=0)
probs_true = probs_true.mean(axis=0)
plt.plot(probs_pred, probs_true, marker='o', linewidth=1, label='Lasso')

line = mlines.Line2D([0, 1], [0, 1], color='black', label='Perfectly Calibrated', linestyle = '--')
transform = ax.transAxes
line.set_transform(transform)
ax.add_line(line)
fig.suptitle('Calibration plot')
ax.set_xlabel('Predicted probability')
ax.set_ylabel('True probability in each bin')
plt.legend()
# plt.show()
plt.savefig(f'../../results/lasso/combined/lasso-cal-curve.png')
plt.close()
# -----------------------------------------------------------------------------------------------------------
 
# ROC -------------------------------------------------------------------------------------------------------
tprs = np.array(tprs)
mean_tprs = tprs.mean(axis=0)
std = tprs.std(axis=0)

mean_auc = auc(base_fpr, mean_tprs)
std_auc = np.std(aucs)

tprs_upper = np.minimum(mean_tprs + std, 1)
tprs_lower = mean_tprs - std

plt.figure(figsize=(12, 8))
plt.plot(base_fpr, mean_tprs, 'darkorange', alpha = 0.8, label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),)
plt.fill_between(base_fpr, tprs_lower, tprs_upper, color = 'grey', alpha = 0.2)
plt.plot([0, 1], [0, 1], linestyle = '--', lw = 2, color = 'navy', label = 'Chance', alpha= 0.8)
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc="lower right")
plt.title('Receiver operating characteristic (ROC) curve')
#plt.axes().set_aspect('equal', 'datalim')
# plt.show()
plt.savefig(f'../../results/lasso/combined/lasso-roc.png')
plt.close()
# ------------------------------------------------------------------------------------------------------------


elapsed_time = (time.time() - start_time)
f.write(f'\nElapsed time: {elapsed_time} seconds \n')

f.write(f'Mean accuracy: \n {mean_accuracy}')
f.write(f'\nMean balanced accuracy:  \n {mean_balanced_accuracy}')
f.write(f'\nMean F1 score: \n {mean_f1}')
f.write(f'\nMean recall: \n {mean_recall}')
f.write(f'\nMean MCC: \n {mean_mcc}')

accuracy_ncv = np.ravel(accuracy_ncv)
balanced_accuracy_ncv = np.ravel(balanced_accuracy_ncv)
f1_ncv = np.ravel(f1_ncv)
recall_ncv = np.ravel(recall_ncv)
mcc_ncv = np.ravel(mcc_ncv)
# f.write(f'\nACC outer folds all runs: {accuracy_ncv}')

mean_accuracy_ncv = str("{:.2f}".format(np.mean(np.array(accuracy_ncv)))) +  " +/- " + str("{:.2f}".format(np.std(np.array(accuracy_ncv))))
f.write(f'\nMean accuracy across all folds: {mean_accuracy_ncv}') # all outer folds accs averaged

mean_balanced_accuracy_ncv = str("{:.2f}".format(np.mean(np.array(balanced_accuracy_ncv)))) +  " +/- " + str("{:.2f}".format(np.std(np.array(balanced_accuracy_ncv))))
f.write(f'\nMean balanced accuracy across all folds: {mean_balanced_accuracy_ncv}') 

mean_f1_ncv = str("{:.2f}".format(np.mean(np.array(f1_ncv)))) +  " +/- " + str("{:.2f}".format(np.std(np.array(f1_ncv))))
f.write(f'\nMean F1 across all folds: {mean_f1_ncv}') 

mean_recall_ncv = str("{:.2f}".format(np.mean(np.array(recall_ncv)))) +  " +/- " + str("{:.2f}".format(np.std(np.array(recall_ncv))))
f.write(f'\nMean recall across all folds: {mean_recall_ncv}') 

mean_mcc_ncv = str("{:.2f}".format(np.mean(np.array(mcc_ncv)))) +  " +/- " + str("{:.2f}".format(np.std(np.array(mcc_ncv))))
f.write(f'\nMean MCC across all folds: {mean_mcc_ncv}') 

sensitivity_ncv = np.array(sensitivity_ncv)
specificity_ncv = np.array(specificity_ncv)
sensitivity_ncv = np.mean(sensitivity_ncv)
specificity_ncv = np.mean(specificity_ncv)

f.write(f'\nMean sensitivity all folds: {round(sensitivity_ncv*100, 2)}%') 
f.write(f'\nMean specificity all folds: {round(specificity_ncv*100, 2)}%') 

In [ ]:
print("Done")
f.close()

# External Test

In [ ]:
df_ext = pd.read_csv("../../data/harmonized/combined_external_harmonized.csv", sep= "\t")
df_ext2 = pd.read_csv("../../data/clinical/clinical_data_external_combined.csv", sep=";", index_col=0)

df_ext2['ID_intern'].replace("LIP", "", regex=True, inplace=True)
df_ext2.drop(df_ext2.columns[1:6], axis=1, inplace=True)
df_ext2.drop(df_ext2.columns[2], axis=1, inplace=True)

df_ext = df_ext2.join(df_ext.set_index('ID_intern'), on='ID_intern')

df_ext = df_ext.rename({'Pathology': 'Type'}, axis=1)

features_ext = list(df_ext.columns[2:])

y_ext = df_ext['Type']
y_ext = pd.DataFrame.to_numpy(y_ext) # data object: numpy array

X_ext = df_ext.drop(df_ext.columns[[0, 1]], axis=1)
X_ext = pd.DataFrame.to_numpy(X_ext) # data object: numpy array

In [ ]:
# CV to test on external dataset
K=3

cv = StratifiedKFold(n_splits=K, random_state=42, shuffle=True)

hp = []
feat = []

# ROC -------------------------------
tprs = []
aucs = []
base_fpr = np.linspace(0, 1, 101)
colors = ['darkorange']

fig, ax = plt.subplots()
# -----------------------------------

# calibration curve -----------------
probs_true = []
probs_pred = []
# -----------------------------------


for train_indices, test_indices in cv.split(X, y): 
    x_train, x_test = X[train_indices], X[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]
    
    accuracies = []
    
    ct = 0
    for item in hyperparams:
        ct = ct + 1
        
        X_train = pd.DataFrame(data=x_train, columns=features)
        X_test = pd.DataFrame(data=x_test, columns=features)
        
        # print("Params: ", item)
        # print("Features: ", hp_features[ct - 1])
        
        cols = [col for col in X_train.columns if col in hp_features[ct - 1]]
        # print(f'\n#common features for {ct-1}: {len(set(hp_features[ct - 1]) & set(cols))}')
        
        X_train = X_train[cols]
        X_test = X_test[cols]
        
        X_train = pd.DataFrame.to_numpy(X_train)
        X_test = pd.DataFrame.to_numpy(X_test)
    
        model.alpha = item
        model.fit(X_train, y_train) 
        
        y_predicted = model.predict(X_test)
        
        
        # ROC ------------------------------------------------------------------------------------------------
        y_score = model.predict_proba(X_test)
        fpr, tpr, _ = roc_curve(y_test, y_score[:, 1])
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        #plt.plot(fpr, tpr, lw=1, alpha=0.6, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc), c = colors[i])
        tpr = interp(base_fpr, fpr, tpr)
        tpr[0] = 0.0
        tprs.append(tpr)
        # ----------------------------------------------------------------------------------------------------
        
        # calibration curve ----------------------------------------------------------------------------------
        prob_true, prob_pred = calibration_curve(y_test, y_score[:,1], strategy='quantile')
        probs_true.append(prob_true)
        probs_pred.append(prob_pred)
        # ----------------------------------------------------------------------------------------------------

        accuracies.append(balanced_accuracy_score(y_test, y_predicted))
         
        hp.append(item)
        feat.append(hp_features[ct - 1])
             
best_accuracy = np.amax(accuracies)
best_hp = hp[np.argmax(accuracies)]
best_features = feat[np.argmax(accuracies)]
    
X = pd.DataFrame(data=X, columns=features)
X_ext = pd.DataFrame(data=X_ext, columns=features)

cols = [col for col in X.columns if col in best_features]

X = X[cols]
X_ext = X_ext[cols]

X = pd.DataFrame.to_numpy(X)
X_ext = pd.DataFrame.to_numpy(X_ext)

model.alpha = best_hp

model.fit(X, y)

y_predicted_ext = model.predict(X_ext)

# calibration curve -----------------------------------------------------------------------------------------
# fig, ax = plt.subplots()
# probs_pred = np.array(probs_pred)
# probs_true = np.array(probs_true)
# probs_pred = probs_pred.mean(axis=0)
# probs_true = probs_true.mean(axis=0)
# plt.plot(probs_pred, probs_true, marker='o', linewidth=1, label='Lasso')
# 
# line = mlines.Line2D([0, 1], [0, 1], color='black', label='Perfectly Calibrated', linestyle = '--')
# transform = ax.transAxes
# line.set_transform(transform)
# ax.add_line(line)
# fig.suptitle('Calibration plot')
# ax.set_xlabel('Predicted probability')
# ax.set_ylabel('True probability in each bin')
# plt.legend()
# # plt.show()
# plt.savefig(f'../../results/lasso/combined/lasso-cal-curve-external.png')
# plt.close()
# # -----------------------------------------------------------------------------------------------------------
#  
# # ROC -------------------------------------------------------------------------------------------------------
# tprs = np.array(tprs)
# mean_tprs = tprs.mean(axis=0)
# std = tprs.std(axis=0)
# 
# mean_auc = auc(base_fpr, mean_tprs)
# std_auc = np.std(aucs)
# 
# tprs_upper = np.minimum(mean_tprs + std, 1)
# tprs_lower = mean_tprs - std
# 
# plt.figure(figsize=(12, 8))
# plt.plot(base_fpr, mean_tprs, 'darkorange', alpha = 0.8, label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),)
# plt.fill_between(base_fpr, tprs_lower, tprs_upper, color = 'grey', alpha = 0.2)
# plt.plot([0, 1], [0, 1], linestyle = '--', lw = 2, color = 'navy', label = 'Chance', alpha= 0.8)
# plt.xlim([-0.01, 1.01])
# plt.ylim([-0.01, 1.01])
# plt.ylabel('True Positive Rate')
# plt.xlabel('False Positive Rate')
# plt.legend(loc="lower right")
# plt.title('Receiver operating characteristic (ROC) curve')
# #plt.axes().set_aspect('equal', 'datalim')
# # plt.show()
# plt.savefig(f'../../results/lasso/combined/lasso-roc-external.png')
# plt.close()
# # ------------------------------------------------------------------------------------------------------------
# 
# 
# 
# # learning curve ----------------------------------------------------------------------------------------------
# train_sizes, train_scores, test_scores, fit_times, _ = learning_curve(
#             model,
#             X,
#             y,
#             cv=cv,
#             n_jobs=4,
#             train_sizes=np.linspace(0.1, 1.0, 5),
#             return_times=True,
#         )
#         
# train_scores_mean = np.mean(train_scores, axis=1)
# train_scores_std = np.std(train_scores, axis=1)
# test_scores_mean = np.mean(test_scores, axis=1)
# test_scores_std = np.std(test_scores, axis=1)
# fit_times_mean = np.mean(fit_times, axis=1)
# fit_times_std = np.std(fit_times, axis=1) 
# 
# 
# plot_learning_curve(train_sizes, train_scores_mean, train_scores_std, test_scores_mean, test_scores_std, 
#                     fit_times_mean, fit_times_std, cv=cv, n_jobs=4)
# plt.savefig(f'../../results/lasso/combined/ncv-lasso-learning-curve-external.png')
# plt.close()
# ------------------------------------------------------------------------------------------------------------


cm = confusion_matrix(y_ext, y_predicted_ext)
plotCM(cm, "", "external", "", "" , 0)
# print("!!!!", cm_o[0,0], cm_o[0,1], cm_o[1,0], cm_o[1,1])

f.write(f'\n\nEXTERNAL TEST METRICS')

sensitivity = cm[1,1]/(cm[1,1]+cm[1,0]) # TP/(TP+TN)
f.write(f'\nSensitivity: {round(sensitivity*100, 2)}%')
specificity = cm[0,0]/(cm[0,0]+cm[0,1]) # TN/(TN+FP)
f.write(f'\nSpecificity: {round(specificity*100, 2)}%')

    
# metrics
accuracy_external = accuracy_score(y_ext, y_predicted_ext)
balanced_accuracy_external = balanced_accuracy_score(y_ext, y_predicted_ext)
f1_external = f1_score(y_ext, y_predicted_ext)
recall_external = recall_score(y_ext, y_predicted_ext)
mcc_external = matthews_corrcoef(y_ext, y_predicted_ext)

f.write(f'\n\nAccuracy: {accuracy_external}')
f.write(f'\nBalanced accuracy: {balanced_accuracy_external}')
f.write(f'\nF1 score: {f1_external}')
f.write(f'\nRecall score: {recall_external}')
f.write(f'\nMCC: {mcc_external}')
f.close()

In [ ]:
print(f'\nSensitivity: {round(sensitivity*100, 2)}%')
print(f'\nSpecificity: {round(specificity*100, 2)}%')

print(f'\n\nAccuracy: {accuracy_external}')
print(f'\nBalanced accuracy: {balanced_accuracy_external}')
print(f'\nF1 score: {f1_external}')
print(f'\nRecall score: {recall_external}')
print(f'\nMCC: {mcc_external}')

# Probabilities - INTERNAL

In [9]:
cases = []

In [10]:
series = range(1, 51)
for i in series:
    cv_outer = StratifiedKFold(n_splits=3, random_state=i, shuffle=True)
    for train_indices, test_indices in cv_outer.split(X, y):
        # print(i, ": " + "Train indices: ", train_indices, "\nTest indices: ", test_indices, "\n\n")
        X_train_i, X_test_i = X[train_indices], X[test_indices]
        cases.append(test_indices)
        y_train_i, y_test_i = y[train_indices], y[test_indices]

In [11]:
# load models
models = []
h = open('/Users/dianadavid/Uni/Thesis/repo/results/lasso/combined/lasso_final_models_combined.dat', 'rb')
for item in range(150):
    models.append(pickle.load(h))
h.close()

In [12]:
file = open('/Users/dianadavid/Uni/Thesis/repo/params/lasso/lasso_features_combined.txt', 'r')
content = file.read()
feats = content.split("\n")
file.close()
feats.pop()
len(feats)

150

In [13]:
hp_features = []
for item in feats:
    my_list = item.split("', '")
    my_list[0] = my_list[0][2:]
    my_list[-1] = my_list[-1][:-2]
    hp_features.append(my_list)
len(hp_features)

150

In [14]:
file = open('/Users/dianadavid/Uni/Thesis/repo/params/lasso/lasso_hyperparams_combined.txt', 'r')
content = file.read()
hyperp = content.split("\n")
file.close()
hyperp.pop()
len(hyperp)

150

In [15]:
import ast
hyperparameters = []
for item in hyperp:
    my_dict = ast.literal_eval(item)
    hyperparameters.append(my_dict)
len(hyperparameters)

150

In [16]:
c = 0
predictions_internal = []
probabilities_internal = []
for model in models:
    x = pd.DataFrame(data=X[cases[c]], columns=features)
    
    # print("Params: ", item)
    # print("Features: ", hp_features[ct - 1])
    
    cols = [col for col in x.columns if col in hp_features[c]]
    
    x = x[cols]
    x = pd.DataFrame.to_numpy(x)

    predictions_internal.append(model.predict(x))
    probabilities_internal.append(model.predict_proba(x))
    
    c = c + 1

In [27]:
cases

[array([  2,   4,   8,   9,  13,  18,  24,  28,  35,  40,  53,  55,  57,
         59,  60,  62,  63,  65,  70,  74,  75,  76,  79,  81,  83,  84,
         88,  91,  98, 100, 102, 104, 107, 110, 113, 116, 119, 126, 127,
        128, 129, 131, 133, 138, 139, 143, 144, 147, 149, 151, 152, 154]),
 array([  0,   3,   7,  10,  11,  12,  19,  20,  31,  32,  33,  34,  36,
         39,  41,  44,  46,  47,  49,  50,  51,  52,  54,  58,  64,  66,
         69,  71,  72,  85,  86,  89,  90,  97,  99, 105, 108, 109, 112,
        114, 115, 117, 124, 134, 136, 137, 141, 142, 145, 146, 148, 150]),
 array([  1,   5,   6,  14,  15,  16,  17,  21,  22,  23,  25,  26,  27,
         29,  30,  37,  38,  42,  43,  45,  48,  56,  61,  67,  68,  73,
         77,  78,  80,  82,  87,  92,  93,  94,  95,  96, 101, 103, 106,
        111, 118, 120, 121, 122, 123, 125, 130, 132, 135, 140, 153]),
 array([  5,   6,   9,  10,  12,  13,  15,  16,  17,  21,  22,  23,  24,
         25,  29,  31,  34,  39,  41,  47,  48,  5

In [18]:
# predictions_internal

In [19]:
probabilities_internal

[array([[9.99974283e-01, 2.57169091e-05],
        [9.99999992e-01, 8.08387605e-09],
        [8.16791435e-01, 1.83208565e-01],
        [1.00000000e+00, 1.27168337e-14],
        [5.52731783e-02, 9.44726822e-01],
        [9.99999615e-01, 3.84693401e-07],
        [1.00000000e+00, 2.57488766e-13],
        [1.00000000e+00, 1.59685425e-19],
        [1.66993585e-01, 8.33006415e-01],
        [9.99999999e-01, 1.13521953e-09],
        [7.59198842e-02, 9.24080116e-01],
        [9.99999999e-01, 8.38303608e-10],
        [1.00000000e+00, 3.07042533e-21],
        [1.00000000e+00, 1.47226836e-21],
        [1.00000000e+00, 4.97639826e-15],
        [1.00000000e+00, 2.26887642e-15],
        [1.00000000e+00, 3.14551407e-11],
        [9.99999999e-01, 9.33632995e-10],
        [9.99999999e-01, 5.95373522e-10],
        [1.00000000e+00, 3.28267732e-16],
        [7.06101844e-14, 1.00000000e+00],
        [9.87321301e-01, 1.26786988e-02],
        [1.00000000e+00, 2.89238829e-11],
        [1.00000000e+00, 2.5999912

In [21]:
# preds = [item for sublist in predictions_internal for item in sublist]
# preds

In [22]:
prob0 = []
prob1 = []

for item in probabilities_internal:
    for it in item:
        prob0.append(it[0]*100)
        prob1.append(it[1]*100)

In [24]:
prob1

[0.002571690905610407,
 8.083876051216297e-07,
 18.320856491672245,
 1.2716833679867664e-12,
 94.47268216961706,
 3.846934011773043e-05,
 2.5748876569196986e-11,
 1.5968542510428856e-17,
 83.30064149979421,
 1.1352195327369214e-07,
 92.4080115816384,
 8.383036080474565e-08,
 3.070425333338945e-19,
 1.4722683626812368e-19,
 4.976398264751599e-13,
 2.268876419829804e-13,
 3.1455140703679117e-09,
 9.336329951064214e-08,
 5.953735223098803e-08,
 3.28267731523963e-14,
 99.99999999999294,
 1.2678698824499117,
 2.892388285325623e-09,
 2.599991270892847e-15,
 94.61280789355477,
 3.471306660226194e-07,
 17.001981354916314,
 1.118194038715389e-08,
 1.0217618157498207e-09,
 1.3098101664702819e-11,
 1.3095479896458378e-10,
 75.72406748004613,
 1.0032213573570337e-15,
 1.3863489140695282e-15,
 0.02807857446454804,
 0.0012210286175305097,
 1.6727726941252597e-22,
 1.1191356533548242e-13,
 7.630049780347755e-13,
 7.212326489427237e-06,
 5.026907565456217e-13,
 99.99999999435532,
 2.360611073144458e-0

In [25]:
cases_all = [item for sublist in cases for item in sublist]

In [26]:
# Calling DataFrame constructor after zipping
# both lists, with columns specified
df = pd.DataFrame(list(zip(cases_all, prob1)),
               columns =['Index', 'Prob1'])
df

,Index,Prob1
0,2,2.571691e-03
1,4,8.083876e-07
2,8,1.832086e+01
3,9,1.271683e-12
4,13,9.447268e+01
...,...,...
7745,144,1.741398e+01
7746,145,8.152460e+00
7747,148,1.055978e+01
7748,150,5.696899e+00


In [28]:
probs = dict()
for index in range(0, 155):
    df_index = df.loc[df['Index'] == index]
    probs[index] = df_index.mean(axis=0)['Prob1']
probs

{0: 48.55559149644899,
 1: 44.35964513082263,
 2: 7.135971060264556,
 3: 83.69164523683885,
 4: 9.734059319474559,
 5: 33.61030021312261,
 6: 9.65304097249807,
 7: 2.896461768200146,
 8: 48.49154427256806,
 9: 3.137019519045938,
 10: 55.53542573853741,
 11: 14.446523615434817,
 12: 3.541650537630667,
 13: 52.57834626004878,
 14: 28.535219645166993,
 15: 5.438014930306678,
 16: 13.920002830551928,
 17: 19.172423026266085,
 18: 11.024120544604877,
 19: 93.5402761482161,
 20: 57.24107185477365,
 21: 19.832953876168386,
 22: 50.73052999427578,
 23: 87.54991274271002,
 24: 5.798065237850611,
 25: 7.622478446343622,
 26: 16.502310763653306,
 27: 65.22838813219066,
 28: 4.005206933242485,
 29: 30.147474054057387,
 30: 34.45173886613893,
 31: 10.637825789333476,
 32: 13.407822737403187,
 33: 5.71260217667255,
 34: 1.7913017440465573,
 35: 28.94478462132486,
 36: 33.684962938625056,
 37: 5.973928434513502,
 38: 22.51786467389656,
 39: 22.979156247618185,
 40: 18.470487314802444,
 41: 10.5419016

In [29]:
# Calling DataFrame constructor after zipping
# both lists, with columns specified
df = pd.DataFrame(list(zip(tumor_ids, probs.values())),
               columns =['Tumor ID', 'Prob1'])
df

,Tumor ID,Prob1
0,LIPO_002,48.555591
1,LIPO_006,44.359645
2,LIPO_007,7.135971
3,LIPO_009,83.691645
4,LIPO_010,9.734059
...,...,...
150,LT202,2.735271
151,LT203,1.741045
152,LT208,35.998118
153,LT214,4.220377


In [30]:
df.to_csv('/Users/dianadavid/Uni/Thesis/repo/results/lasso/combined/lasso-probs-internal.csv', index=False)

# Probabilities - EXTERNAL

In [ ]:
df_ext = pd.read_csv("../../data/harmonized/combined_external_harmonized.csv", sep= "\t")
df_ext2 = pd.read_csv("../../data/clinical/clinical_data_external_combined.csv", sep=";", index_col=0)

df_ext2['ID_intern'].replace("LIP", "", regex=True, inplace=True)
df_ext2.drop(df_ext2.columns[1:6], axis=1, inplace=True)
df_ext2.drop(df_ext2.columns[2], axis=1, inplace=True)

df_ext = df_ext2.join(df_ext.set_index('ID_intern'), on='ID_intern')

df_ext = df_ext.rename({'Pathology': 'Type'}, axis=1)

cases = df_ext['ID_intern']

features_ext = list(df_ext.columns[2:])

y_ext = df_ext['Type']
y_ext = pd.DataFrame.to_numpy(y_ext) # data object: numpy array

X_ext = df_ext.drop(df_ext.columns[[0, 1]], axis=1)
X_ext = pd.DataFrame.to_numpy(X_ext) # data object: numpy array

In [ ]:
k = open('/Users/dianadavid/Uni/Thesis/repo/results/lasso/combined/lasso_model_combined.dat', 'rb')

In [ ]:
model = pickle.load(k)

In [ ]:
model

In [ ]:
import ast
file = open('../../params/lasso/lasso_best_features_combined.txt', 'r')
content = file.read()
feats = content.split("\n")
file.close()
feats.pop()
# print(feats)

In [ ]:
hp_features = feats
# for item in feats:
#     my_list = item.split("', '")
#     my_list[0] = my_list[0][2:]
#     my_list[-1] = my_list[-1][:-2]
#     hp_features.append(my_list)
# print(hp_features)

In [ ]:
file = open('../../params/lasso/lasso_best_hyperparams_combined.txt', 'r')
content = file.read()
hyperp = content.split("\n")
file.close()
hyperp.pop()
# print(hyperp)

In [ ]:
hyperparams = []
for item in hyperp:
    my_dict = ast.literal_eval(item)
    hyperparams.append(my_dict)
# print(hyperparams)

In [ ]:
x = pd.DataFrame(data=X, columns=features)
x_ext = pd.DataFrame(data=X_ext, columns=features)

# print("Params: ", item)
# print("Features: ", hp_features)

cols = [col for col in x.columns if col in hp_features]

x = x[cols]
x_ext = x_ext[cols]

x = pd.DataFrame.to_numpy(x)
x_ext = pd.DataFrame.to_numpy(x_ext)

In [ ]:
predictions = model.predict(x_ext)
probabilities = model.predict_proba(x_ext)

In [ ]:
predictions

In [ ]:
# probabilities

In [ ]:
probabilities[0,0] + probabilities[0,1]

In [ ]:
prob0 = []
prob1 = []

for item in probabilities:
    prob0.append(round(item[0]*100,2))
    prob1.append(round(item[1]*100,2))

In [ ]:
# Calling DataFrame constructor after zipping
# both lists, with columns specified
df = pd.DataFrame(list(zip(cases, predictions, prob0, prob1)),
               columns =['Tumor ID', 'Actual pred', 'Prob0', 'Prob1'])
df

In [ ]:
df.to_csv('/Users/dianadavid/Uni/Thesis/repo/results/lasso/combined/lasso-probs.csv', index=False)